# Ages and Masses from GALAH Spectra with The Cannon

## Part 1: Creating a training set

In [1]:
# Preamble
try:
    %matplotlib inline
    %config InlineBackend.figure_format='retina'
except:
    pass

from astropy.io import fits
import numpy as np
from astropy.table import Table, join
from scipy.io.idl import readsav
import astropy.io.fits as pyfits
import matplotlib.pyplot as plt
import pickle

# Read in and join GALAH DR3 and asteroseismic catalogs

In [2]:
# Read in GALAH DR3
galah_dr3 = Table.read('../GALAH_DR3/catalogs/GALAH_DR3_all_joined_v2.fits')
galah_dr3 = galah_dr3[galah_dr3['dr3_source_id_1'] > 0]

In [3]:
# Read in the asteroseismic data from Zinn et al. (2022)
seismic = Table.read('../GALAH_DR4/auxiliary_information/Zinn_Table2_eDR3_xmatch.fits')

# Rename some keywords for ease
seismic['dr3_source_id_1'] = seismic['source_id']
seismic['nu_max'] = seismic['numax_mean']
seismic['delta_nu'] = seismic['dnu_mean_corr']

In [4]:
# Join GALAH DR3 and the asteroseismic data
data = join(galah_dr3, seismic, keys='dr3_source_id_1')

# Rename some keywords for ease
data['mass'] = data['m_act_bstep']
data['age'] = data['age_bstep']

# Calculate masses from scaling relations
data['mass_astero'] = (data['nu_max']/3076.)**3. * (data['delta_nu']/135.146)**(-4) * (data['teff']/5772)**(3./2.)

# Trainingset Label Selection

We have to do 2 steps, because not all spectra are available publicly for GALAH DR3  
So our approach is: select all useful catalog entries, then make sure they all have spectra, then save those as the trainingset

In [5]:
# Select a useful preliminary trainingset

selection = (
    (data['flag_sp'] == 0) & 
    (data['flag_fe_h'] == 0) & 
    (data['flag_alpha_fe'] == 0) & 
    (data['snr_c2_iraf'] > 75) & 
    (data['teff'] < 5150) & 
    np.isfinite(data['age']) & 
    np.isfinite(data['mass']) & 
    np.isfinite(data['numax_mean']) & 
    np.isfinite(data['dnu_mean_corr'])
)
print(len(data['sobject_id'][selection]))

/Users/svenbuder/anaconda3/lib/python3.7/site-packages/astropy/table/column.py:984: RuntimeWarning: invalid value encountered in less
  result = getattr(super(), op)(other)


989


In [6]:
# Create catalog of trainingset
preliminary_trainingset = Table()
for key in ['sobject_id', 'teff', 'logg','fe_h','alpha_fe','nu_max', 'delta_nu','mass','age','mass_astero','j_m','h_m','ks_m']:
    preliminary_trainingset[key] = data[key][selection]

In [7]:
# Save the training set ids, so that we can download them from datacentral.org.au
np.savetxt('trainingset_ids.txt',[", ".join(str(x) for x in preliminary_trainingset['sobject_id'])],fmt='%s')

# Now use this list for a bulk download (as described on galah-survey.org) from https://datacentral.org.au/services/download/

In [8]:
# Extract the spectra in observation/hermes/*sobejctid*ccd*.fits

# Now we check which spectra are missing
missing = []

for sobject_id in preliminary_trainingset['sobject_id']:
    
    try:
        for ccd in [1,2,3,4]:

            fits_file = fits.open('observation/hermes/'+str(sobject_id)+str(ccd)+'.fits')
            s = fits_file[4].data

            fits_file.close()
    except:
        missing.append(sobject_id)

# Vice versa: Which spectra are available?
available = []

for sobject_id in preliminary_trainingset['sobject_id']:
    if sobject_id not in missing:
        available.append(True)
    else:
        available.append(False)

In [9]:
# Now save the final trainingset for which we know all spectra are available

trainingset = preliminary_trainingset[available]
final_trainingset = Table()
for key in ['sobject_id', 'teff', 'logg','fe_h','alpha_fe','nu_max', 'delta_nu','mass','age','mass_astero','j_m','h_m','ks_m']:
    final_trainingset[key] = trainingset[key]
    
final_trainingset.write('trainingset.fits',overwrite=True)
final_trainingset

sobject_id,teff,logg,fe_h,alpha_fe,nu_max,delta_nu,mass,age,mass_astero,j_m,h_m,ks_m
,K,log(cm.s**-2),,,,,,,,mag,mag,mag
int64,float32,float32,float32,float64,float32,float32,float64,float64,float32,float32,float32,float32
161006005401094,4830.7607,2.364735,-0.33675337,0.0800271859202082,32.167,4.337,1.0947040243794466,6.376832077049619,0.82558477,9.194,8.631,8.437
161006005401076,4672.3394,2.3231006,-0.115608215,0.049204728181834044,37.015,4.196,1.1224882870355306,6.593441477004973,1.3657057,8.827,8.198,8.045
140824006301117,4584.031,2.1341462,-0.29746866,0.06269277936057815,35.526,3.929,1.1185212152459802,6.93231590248784,1.5263231,7.334,6.715,6.545
140824006301178,4868.853,2.492118,-0.15996265,0.04322181828260722,89.557,7.399,1.7387946464289636,2.03368636095098,2.1281917,8.63,8.068,7.885
151111003101006,4551.1636,1.9781973,-0.46227074,0.08734366140261351,15.636,2.479,1.079395123830893,7.143227732624391,0.8123041,7.72,7.017,6.845
151111003101382,4220.5767,1.9814734,-0.085291386,0.16184932646030029,18.663,2.508,1.0569430544009966,9.877934263582587,1.1774876,7.365,6.646,6.414
160923004201116,4466.065,2.2739487,-0.011614323,0.10969648273252203,31.858,3.906,1.0849137737086834,8.455245029001855,1.0836236,9.181,8.548,8.365
160923004201056,4579.5615,2.6352112,-0.0041708946,0.03420725546068786,68.692,6.537,1.069706517888882,9.392495598718993,1.4378201,9.318,8.681,8.513


# Trainingset Wavelength, Flux, and InverseVariance Preparation

Now that we have saved all labels, we need to prepare the wavelengths, fluxes, and inverse variance (ivar) values for the training set

In [10]:
# Wavelength array
# For GALAH, we have 4 CCDs

cannon_wave = dict()
cannon_wave['1']=np.arange(4715.94,4896.00,0.046) # ab lines 4716.3 - 4892.3
cannon_wave['2']=np.arange(5653.31,5868.25,0.055) # ab lines 5646.0 - 5867.8
cannon_wave['3']=np.arange(6480.52,6733.92,0.064) # ab lines 6481.6 - 6733.4
cannon_wave['4']=np.arange(7726.95,7875.55,0.074) # ab lines 7691.2 - 7838.5

wavelength_array = np.concatenate(([cannon_wave[key] for key in ['1','2','3','4']]))
np.savetxt('wavelength.txt',wavelength_array,fmt='%s')

In [11]:
# The Cannon needs FLUX and IVAR as inputs
# Lets prepare empty arrays and then fill them by looping through the individual training set entries

normalized_flux = np.ones((np.shape(final_trainingset)[0],np.shape(wavelength_array)[0]))
normalized_ivar = np.ones((np.shape(final_trainingset)[0],np.shape(wavelength_array)[0]))

for index, sobject_id in enumerate(final_trainingset['sobject_id']):
    
    normalised_flux_for_index = []
    normalised_ivar_for_index = []
    
    for ccd in [1,2,3,4]:
        
        fits_file = fits.open('observation/hermes/'+str(sobject_id)+str(ccd)+'.fits')
        wave = fits_file[4].header['CRVAL1'] + fits_file[4].header['CDELT1'] * np.arange(fits_file[4].header['NAXIS1'])
        flux = fits_file[4].data
        flux_unc = fits_file[4].data * fits_file[1].data
        
        # Not all spectra are already on the Cannon wavelength grid, so we have to interpolate them onto the Cannon wavelength grid
        flux_interpolated = np.interp(cannon_wave[str(ccd)], wave, flux)
        flux_unc_interpolated = np.interp(cannon_wave[str(ccd)], wave, flux_unc)
        
        normalised_flux_for_index.append(flux_interpolated)
        normalised_ivar_for_index.append(1./(flux_unc_interpolated**2))
        
    normalised_flux_for_index = np.concatenate((normalised_flux_for_index))
    normalised_ivar_for_index = np.concatenate((normalised_ivar_for_index))
    
    normalized_flux[index] = normalised_flux_for_index
    normalized_ivar[index] = normalised_ivar_for_index
    
# Now save these 2 entries to a pickle file (more efficient storage)
flux_ivar_file_opener = open('trainingset_flux_ivar.pickle','wb')
pickle.dump((normalized_flux,normalized_ivar),flux_ivar_file_opener)
flux_ivar_file_opener.close()